In [5]:
import os
import tweepy
from dotenv import load_dotenv
from textblob import TextBlob
import pandas as pd
import time
from tweepy.errors import TooManyRequests

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener credenciales de la API de Twitter (X) desde las variables de entorno
BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")

# Autenticación con la API v2 de Twitter
client = tweepy.Client(bearer_token=BEARER_TOKEN)

# Función para obtener tweets sobre un tema específico con la API v2
def obtener_tweets(tema, cantidad=10, intentos=4):
    """
    Busca tweets recientes en Twitter usando la API v2, excluyendo retweets y filtrando por idioma español.
    
    Parámetros:
    - tema: str -> La palabra clave o frase a buscar en los tweets.
    - cantidad: int -> Número de tweets a obtener (máximo permitido en una sola solicitud es 100).
    - intentos: int -> Número de intentos antes de esperar nuevamente.

    Retorna:
    - Lista de diccionarios con el texto del tweet, el nombre de usuario y la fecha de publicación.
    """
    # Modificación: Agregamos "-is:retweet" para excluir retweets y "lang:es" para solo obtener tweets en español
    query = f"{tema} -is:retweet lang:es"
    
    for i in range(intentos):
        try:
            tweets = client.search_recent_tweets(query=query, max_results=cantidad, tweet_fields=["author_id", "created_at"])
            if tweets.data:
                # Modificación: Usamos un conjunto para eliminar tweets duplicados basados en su texto
                tweets_unicos = {}
                for tweet in tweets.data:
                    if tweet.text not in tweets_unicos:
                        tweets_unicos[tweet.text] = {"texto": tweet.text, "usuario": tweet.author_id, "fecha": tweet.created_at}
                return list(tweets_unicos.values())
            else:
                return []
        except TooManyRequests:
            print(f"Intento {i+1} fallido: Demasiadas solicitudes. Esperando 10 minutos...")
            time.sleep(600)  # Espera 10 minutos
    print("Se agotaron los intentos, por favor intenta más tarde.")
    return []

# Función para analizar el sentimiento de un texto con TextBlob
def analizar_sentimiento(texto):
    """
    Analiza el sentimiento de un texto utilizando TextBlob.
    
    Parámetros:
    - texto: str -> Texto del tweet.

    Retorna:
    - Un valor entre -1 y 1:
        * -1 significa sentimiento negativo.
        *  0 significa sentimiento neutro.
        *  1 significa sentimiento positivo.
    """
    blob = TextBlob(texto)
    return blob.sentiment.polarity

# Función para obtener el emoji según la polaridad
def obtener_emoji(sentimiento):
    """
    Devuelve un emoji basado en el sentimiento.
    
    Parámetros:
    - sentimiento: float -> Polaridad del sentimiento.

    Retorna:
    - Emoji representativo del sentimiento.
    """
    if sentimiento > 0.1:
        return "😊"  # Positivo
    elif sentimiento < -0.1:
        return "😠"  # Negativo
    else:
        return "😐"  # Neutro

# Función para guardar los tweets en un archivo de texto
def guardar_tweets_en_archivo(tweets, archivo="tweets.txt"):
    with open(archivo, "w", encoding="utf-8") as f:
        for tweet in tweets:
            # Obtiene el sentimiento y el emoji representativo
            sentimiento = analizar_sentimiento(tweet['texto'])
            emoji = obtener_emoji(sentimiento)
            
            # Determina el tipo de sentimiento
            if sentimiento > 0.1:
                sentimiento_tipo = "Positivo"
            elif sentimiento < -0.1:
                sentimiento_tipo = "Negativo"
            else:
                sentimiento_tipo = "Neutro"
            
            # Escribe cada tweet en el formato: @usuario: "texto" - fecha - Sentimiento: <tipo> <emoji>
            f.write(f"@{tweet['usuario']}: \"{tweet['texto']}\" - {tweet['fecha']} - Sentimiento: {sentimiento_tipo} {emoji}\n\n")

# Definir el tema de los tweets a buscar
tema = "motogp"  # Puedes cambiar esta palabra clave a cualquier tema que te interese

# Obtener tweets y analizarlos
tweets = obtener_tweets(tema, 10)

# Si hay tweets, guardarlos en el archivo
if tweets:
    # Guardar los tweets en el archivo
    guardar_tweets_en_archivo(tweets)
    print("Tweets guardados en el archivo.")
else:
    print("No se encontraron tweets para el tema especificado.")

# Mostrar los resultados en forma de DataFrame
df = pd.DataFrame(tweets)

if not df.empty:
    # Aplicar la función de análisis de sentimiento a cada tweet y obtener el emoji
    df["sentimiento"] = df["texto"].apply(analizar_sentimiento)
    df["emoji"] = df["sentimiento"].apply(obtener_emoji)
    
    # Determinar el sentimiento basado en la polaridad
    df["sentimiento_tipo"] = df["sentimiento"].apply(lambda x: "Positivo" if x > 0.1 else ("Negativo" if x < -0.1 else "Neutro"))
    
    # Mostrar el DataFrame con los resultados
    print(df[["texto", "sentimiento_tipo", "emoji"]])
else:
    print("No se encontraron tweets para mostrar.")

Tweets guardados en el archivo.
                                               texto sentimiento_tipo emoji
0  Os dais cuenta de la barbaridad que es esto? h...           Neutro     😐
1  Ni la victoria de Marc Márquez con Ducati enga...           Neutro     😐
2  🗣️ “Arrancar este sueño de Ducati con estas vi...           Neutro     😐
3  Marc Márquez alcanzó en Chang las 10 temporada...           Neutro     😐
4  “Ducati Pramac no es una opción”\n\n- Porque?\...           Neutro     😐
5    @MotoGP_ESPN Ya lo pensaba antes de la carrera.           Neutro     😐
6  @MajoAMartinez JAJAJAJ yo pa la f1 hago eso pe...           Neutro     😐
7  Aldeguer se da un aprobado en su estreno en #M...           Neutro     😐
8  @MotoGP_ESPN Tremendo regalo te hizo Italia un...           Neutro     😐
9  💫 La firma Márquez hace más historia\n\n⚡ Marc...           Neutro     😐
